In [1]:
import os
# Changing the working directory
os.chdir('../..')
os.getcwd()

'/Users/javiermunoz/Universidad/MasterDSyBD/TFM/TFM_DSBD'

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc("figure", figsize=(15, 5))

import numpy as np
import pandas as pd

In [3]:
leaderboard = pd.read_csv("data/leaderboard.csv")
leaderboard

,model_type,model_name,train_acc,val_acc,test_acc,train_loss,val_loss,test_loss,epoch,path_to_model,subset_size
0,convlstm,simple,0.96,0.93,0.93,0.15,0.20,0.19,39.0,src/convlstm/bestmodels/best_simple_39_0.93_10...,10.0
1,convlstm,stateless,0.93,0.95,0.91,0.22,0.19,0.19,22.0,src/convlstm/bestmodels/best_stateless_22_0.95...,10.0
2,mediapipe,mp,0.90,0.90,0.87,0.27,0.35,0.54,178.0,src/mediapipe/bestmodels/best_mp_178_0.90_mode...,20.0
3,mediapipe,mp,0.89,0.85,0.85,0.38,0.62,0.43,159.0,src/mediapipe/bestmodels/best_mp_159_0.85_mode...,40.0
4,mediapipe,mp,0.93,0.86,0.86,0.24,0.63,0.55,167.0,src/mediapipe/bestmodels/best_mp_167_0.86_mode...,30.0
5,3DCNN,3dcnn,0.93,0.92,0.91,0.33,0.37,0.40,132.0,src/3DCNN/bestmodels/best_3dcnn_132_0.92_model.h5,20.0
6,convlstm,stateless,0.90,0.94,0.92,0.27,0.24,0.22,63.0,src/convlstm/bestmodels/best_stateless_63_0.94...,20.0
7,convlstm,stateless,0.95,0.92,0.82,0.20,0.26,0.46,93.0,src/convlstm/bestmodels/best_stateless_91_0.92...,30.0
8,convlstm,stateless,0.87,0.77,0.74,0.38,1.05,1.06,104.0,src/convlstm/bestmodels/best_stateless_104_0.7...,40.0
9,convlstm,simple,0.83,0.65,0.65,0.48,1.36,1.27,170.0,src/convlstm/bestmodels/best_simple_170_0.65_m...,40.0


In [3]:
# Define the dimensions of one frame in the set of frames created
HEIGHT = 224
WIDTH = 224

from src.frame_generator import FrameGenerator
from blocks import Conv2Plus1D, ResizeVideo, add_residual_block
from src.config import DATA_PATH, SPLITS
from src.plot_utils import plot_history
import numpy as np
from tensorflow import keras
from keras import layers

input_shape = (None, 10, HEIGHT, WIDTH, 3)
input = layers.Input(shape=(input_shape[1:]))
x = input

x = Conv2Plus1D(filters=16, kernel_size=(3, 7, 7), padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = ResizeVideo(HEIGHT // 2, WIDTH // 2)(x)

# Block 1
x = add_residual_block(x, 16, (3, 3, 3))
x = ResizeVideo(HEIGHT // 4, WIDTH // 4)(x)

# Block 2
x = add_residual_block(x, 32, (3, 3, 3))
x = ResizeVideo(HEIGHT // 8, WIDTH // 8)(x)

# Block 3
x = add_residual_block(x, 64, (3, 3, 3))
x = ResizeVideo(HEIGHT // 16, WIDTH // 16)(x)

# Block 4
x = add_residual_block(x, 128, (3, 3, 3))

x = layers.GlobalAveragePooling3D()(x)
x = layers.Flatten()(x)
x = layers.Dense(10)(x)

model_to_test = keras.Model(input, x)

ModuleNotFoundError: No module named 'einops'

In [ ]:
model_to_test.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              optimizer = keras.optimizers.Adam(learning_rate = 0.0001), 
              metrics = ['accuracy'])

In [ ]:
model_to_test.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10, 224, 22  0           []                               
                                4, 3)]                                                            
                                                                                                  
 conv2_plus1d (Conv2Plus1D)     (None, 10, 224, 224  3152        ['input_1[0][0]']                
                                , 16)                                                             
                                                                                                  
 batch_normalization (BatchNorm  (None, 10, 224, 224  64         ['conv2_plus1d[0][0]']           
 alization)                     , 16)                                                         

In [ ]:
path_to_3dcnn_weights = leaderboard['path_to_model'][3]
model_to_test.load_weights(path_to_3dcnn_weights)

In [ ]:
from src.utils import load_dict
subset = 10
labels=load_dict(f'data/subset_{subset}_lsa_64/pickl_files/labels_map.pkl')
labels

{'Barbecue': 0,
 'Birthday': 1,
 'Buy': 2,
 'Chewing-gum': 3,
 'Coin': 4,
 'Milk': 5,
 'Mock': 6,
 'Realize': 7,
 'Sweet milk': 8,
 'To land': 9}

## Inference based on video

In [5]:
import cv2
from src.load_dataset_aux import format_frames
from numpy.random import default_rng

rng = default_rng()
threshold = 0.7
output_size = (224,224)

cap = cv2.VideoCapture('/home/javiermunoz/TFM_DSBD/data/subset_10_lsa_64/test/Mock/029_001_004.mp4')

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

captured_frames=[]
count=1 
ret=True
while ret:
    ret, frame = cap.read()
    captured_frames.append(frame)

cap.release()
cv2.destroyAllWindows()
    
frames2extract_idx = np.array(sorted(rng.choice(len(captured_frames), size=10, replace=False)))

frames_to_predict = []
for i in range(len(captured_frames)):
    if i in frames2extract_idx:
        frames_to_predict.append(captured_frames[i])

result = []
for frame in frames_to_predict:
    frame_formatted = format_frames(frame, output_size)
    result.append(frame_formatted)

result = np.array(result)[..., [2,1,0]]

logits = model_to_test.predict(np.expand_dims(result, axis=0))
pred_probs = tf.nn.softmax(logits, axis=1).numpy()
print(pred_probs)

if pred_probs[pred_probs > threshold].any():
    pred = np.argmax(pred_probs)
    pred_label = [k for k, v in labels.items() if v == pred]
    print(pred_label)

captured_frames = []

OpenCV: Couldn't read video stream from file "/home/javiermunoz/TFM_DSBD/data/subset_10_lsa_64/test/Mock/029_001_004.mp4"


ValueError: Cannot take a larger sample than population when replace is False

## Inference based on video camera

In [6]:
import cv2
from src.load_dataset_aux import format_frames
from numpy.random import default_rng

rng = default_rng()
threshold = 0.7
output_size = (224,224)

cap = cv2.VideoCapture(0)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

captured_frames=[]
count=1 
ret=True
while ret:
    ret, frame = cap.read()
    captured_frames.append(frame)
    if count % 80 == 0:
        frames2extract_idx = np.array(sorted(rng.choice(len(captured_frames), size=10, replace=False)))
        
        frames_to_predict = []
        for i in range(len(captured_frames)):
            if i in frames2extract_idx:
                frames_to_predict.append(captured_frames[i])


        result = []
        for frame in frames_to_predict:
            frame_formatted = format_frames(frame, output_size)
            result.append(frame_formatted)
        
        result = np.array(result)[..., [2,1,0]]

        logits = model_to_test.predict(np.expand_dims(result, axis=0))
        pred_probs = tf.nn.softmax(logits, axis=1).numpy()
        print(pred_probs)
        if pred_probs[pred_probs > threshold].any():
            pred = np.argmax(pred_probs)
            pred_label = [k for k, v in labels.items() if v == pred]
            print(pred_label)

        captured_frames = []

    count += 1
    # Show to screen
    cv2.imshow('OpenCV Feed', frame)

    # Break gracefully
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


NameError: name 'model_to_test' is not defined